In [26]:
import numpy as np
import csv
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
from Bio.PDB import *
from Bio import SeqIO
import nglview as nv
from glob import glob

In [11]:
"""
Collect pdb data files in the pdbFiles folder.
Then use Biopython to parse the pdb files and get the structure data.
Append that structure data to the pdbs list for processing later.

This was the idea although most of the processing may be just done in the functions rather than stored in a list to save memory.
Also certain processes require more than the Biopython structure object since it cannot store the entirety of the data in the pdb
file.
"""

pdbs = []
parser = PDBParser()
for f in glob('pdbFiles/*.pdb'):
    pdbs.append(parser.get_structure(f'{f}',f))

view = nv.show_biopython(pdbs[0][0])

print(pdbs[0][0]['A'].get_id())

C:\Users\chekm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 21843.
  warnings.warn(
C:\Users\chekm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 21857.
  warnings.warn(
C:\Users\chekm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 21869.
  warnings.warn(
C:\Users\chekm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is di

A


In [13]:
def getBindingRegionResidues(chain1, chain2):
    # Convert the chain objects to lists of atoms
    atoms1 = list(chain1.get_atoms())
    atoms2 = list(chain2.get_atoms())
    
    # Create a NeighborSearch object for chain1
    ns1 = NeighborSearch(atoms1)
    
    # Find the residues in chain2 that are within 4 angstroms of chain1
    nearby_residues2 = []
    residue = None
    neighbors = None
    for atom in atoms2:
        neighbors = ns1.search(atom.coord, 4)
        for neighbor in neighbors:
            residue = neighbor.get_parent()
            if residue not in nearby_residues2:
                nearby_residues2.append(residue)
    
    # Create a NeighborSearch object for chain2
    ns2 = NeighborSearch(atoms2)

    # Find the residues in chain1 that are within 4 angstroms of chain2
    nearby_residues1 = []
    residue = None
    neighbors = None
    for atom in atoms1:
        neighbors = ns2.search(atom.coord, 4)
        for neighbor in neighbors:
            residue = neighbor.get_parent()
            if residue not in nearby_residues1:
                nearby_residues1.append(residue)

    output = {chain1.get_id():nearby_residues1, 
              chain2.get_id():nearby_residues2}
    
    return output

In [14]:
#Get info on the different biological assembly information in the pdb file.
def getBiologicalAssemblies(file):
    pdb = []
    with open(file, 'r') as file:
        for line in file:
            #Crystal structure/Biological Assembly details are stored in REMARK lines 350
            if 'REMARK 350' in line:
                pdb.append(" ".join(line.split()))

    assemblies = {}
    counter = 1
    for line in pdb:
        if "APPLY THE FOLLOWING TO CHAINS" in line:
            assemblies[counter] = line.replace("REMARK 350 APPLY THE FOLLOWING TO CHAINS: ", "").replace(',', "").split()
            counter += 1

    return assemblies

In [15]:
def computeBindingRegion(pdbFile):
    #Get biological assemblies in the structure. This requires to reopen the file and read the REMARK lines.
    BioAssemblies = getBiologicalAssemblies(file = pdbFile)
    
    #Parse the pdb file using Biopython and extract the structure information.
    parser = PDBParser()
    structure = parser.get_structure(id = pdbFile,file = pdbFile)

    #Find what chains are part of protein and antibodies.
    compoundInfo = structure.header['compound']
    for k, v in compoundInfo.items():
        if 'antibody' in v['molecule']:
            if 'heavy' in v['molecule']:
                heavyChains = [c for c in v['chain'].upper().replace(',', "").split()]
            elif 'light' in v['molecule']:
                lightChains = [c for c in v['chain'].upper().replace(',', "").split()]
        else:
            protein = [c for c in v['chain'].replace(',', "").upper().split()]

    #Compute binding residues in the structure for protein/heavy & light chain interactions for each biological assembly.
    chainPairs = {}
    currProtein = None
    for k, v in BioAssemblies.items():
        chainPairs[k] = []
        for c in protein:
            if c in v:
                currProtein = c
                v.remove(currProtein)
        for chain in v:
            chainPairs[k].append((currProtein, chain))

    #Compute binding regions
    chain1 = None
    chain2 = None
    BindingRegion = {}
    for k, v in chainPairs.items():
        for pair in v:
            for chain in structure[0]:  
                if chain.get_id() == pair[0]:
                    chain1 = chain
                elif chain.get_id() == pair[1]:
                    chain2 = chain
            resIdsAtBindReg = getBindingRegionResidues(chain1 = chain1, chain2 = chain2)
            BindingRegion[pair] = resIdsAtBindReg
    return BindingRegion

In [16]:
bind = computeBindingRegion(pdbFile='pdbFiles/7fah.pdb')
#print(bind[('A', 'H')][0].get_id()[1])
print(bind)

{('A', 'H'): {'A': [<Residue TYR het=  resseq=54 icode= >, <Residue ASP het=  resseq=102 icode= >, <Residue ASN het=  resseq=52 icode= >, <Residue THR het=  resseq=55 icode= >, <Residue ASN het=  resseq=32 icode= >, <Residue LYS het=  resseq=31 icode= >, <Residue GLY het=  resseq=33 icode= >, <Residue MET het=  resseq=99 icode= >, <Residue VAL het=  resseq=100 icode= >, <Residue ARG het=  resseq=101 icode= >, <Residue TRP het=  resseq=50 icode= >], 'H': [<Residue ALA het=  resseq=148 icode= >, <Residue ALA het=  resseq=150 icode= >, <Residue HIS het=  resseq=147 icode= >, <Residue GLU het=  resseq=77 icode= >, <Residue GLY het=  resseq=149 icode= >, <Residue LYS het=  resseq=151 icode= >, <Residue ALA het=  resseq=143 icode= >]}, ('A', 'L'): {'A': [<Residue ASP het=  resseq=32 icode= >, <Residue PHE het=  resseq=31 icode= >, <Residue GLU het=  resseq=97 icode= >, <Residue TYR het=  resseq=36 icode= >, <Residue TYR het=  resseq=34 icode= >, <Residue SER het=  resseq=95 icode= >, <Residu

C:\Users\chekm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 21843.
  warnings.warn(
C:\Users\chekm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 21857.
  warnings.warn(
C:\Users\chekm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 21869.
  warnings.warn(
C:\Users\chekm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is di

In [41]:
#Format the data and label residues participating in binding.
def labelBindingResidues(pdbFile, outputFile):
    #Ignore warnings
    warnings.simplefilter('ignore', PDBConstructionWarning)
    BindingRegion = computeBindingRegion(pdbFile = pdbFile)

    #Consolidate Binding residue sequence IDs with their chain.
    BindingDict = {}
    for k, v in BindingRegion.items():
        for chain, residues in v.items():
            if chain not in BindingDict.keys():
                BindingDict[chain] = []
            for residue in residues:
                BindingDict[chain].append(residue.get_id()[1])

    #Get the sequence for chains.
    for record in SeqIO.parse(pdbFile, "pdb-seqres"):
        print(record)
    
    '''
    SequenceData = {record.annotations["chain"]: str(record.seq) for record in SeqIO.parse(pdbFile, "pdb-seqres")}
    print(SequenceData)
    data = []
    for chain, residues in SequenceData:
        for i in len(residues):
            data.append([
                chain, i+1, residues[i]
            ])
    '''
    #Create table
    return BindingDict

In [42]:
print(labelBindingResidues(pdbFile = 'pdbFiles/7fah.pdb', outputFile = "7fah_binding.csv"))

ID: 7FAH:A
Name: 7FAH:A
Description: UNP:C3W5S1 C3W5S1_I09A0
Database cross-references: UNP:C3W5S1, UNP:C3W5S1_I09A0
Number of features: 0
/chain=A
/molecule_type=protein
Seq('MDTLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGVAPLHLG...HHH')
ID: 7FAH:B
Name: 7FAH:B
Description: UNP:C3W5S1 C3W5S1_I09A0
Database cross-references: UNP:C3W5S1, UNP:C3W5S1_I09A0
Number of features: 0
/chain=B
/molecule_type=protein
Seq('MDTLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKHNGKLCKLRGVAPLHLG...HHH')
ID: 7FAH:C
Name: 7FAH:C
Description: PDB:7FAH 7FAH
Database cross-references: PDB:7FAH, PDB:7FAH
Number of features: 0
/chain=C
/molecule_type=protein
Seq('QIQLVQSGPELKKPGETVRISCKASGYTFTKNGMNWVQQAPGKGLKWVGWINTY...VPR')
ID: 7FAH:D
Name: 7FAH:D
Description: PDB:7FAH 7FAH
Database cross-references: PDB:7FAH, PDB:7FAH
Number of features: 0
/chain=D
/molecule_type=protein
Seq('DIVLTQSPASLAVSLGQRATISCKASQSVDFDGYNYLNWYQQKPGQPPKLLIYA...NEC')
ID: 7FAH:H
Name: 7FAH:H
Description: PDB:7FAH 7FAH
Database cross-references: PDB:7F